In [4]:
import sys
sys.path.append('./aienginepackages')

import pandas as pd
from aienginepackages import process_data, determine_feature_set

In [5]:
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import pandas as pd

In [8]:
df = pd.read_csv('datasets/modifiedkaggletraffic2.csv', names=['date_time', 'value','id'])

In [9]:
proc_data = process_data(df, quick_start=False)

In [11]:
proc_data.dataset.tail()

,value,id
date_time,,
2017-06-30 19:00:00,105.0,20170630191
2017-06-30 20:00:00,96.0,20170630201
2017-06-30 21:00:00,90.0,20170630211
2017-06-30 22:00:00,84.0,20170630221
2017-06-30 23:00:00,78.0,20170630231


In [24]:
proc_data.dataset.index.values

array(['2015-11-01T00:00:00.000000000', '2015-11-01T01:00:00.000000000',
       '2015-11-01T02:00:00.000000000', ...,
       '2017-06-30T21:00:00.000000000', '2017-06-30T22:00:00.000000000',
       '2017-06-30T23:00:00.000000000'], dtype='datetime64[ns]')

In [22]:
proc_data.dataset.dtypes

value    float64
id         int64
dtype: object

In [25]:
proc_data.dataset['value']

date_time
2015-11-01 00:00:00     15.0
2015-11-01 01:00:00     13.0
2015-11-01 02:00:00     10.0
2015-11-01 03:00:00      7.0
2015-11-01 04:00:00      9.0
                       ...  
2017-06-30 19:00:00    105.0
2017-06-30 20:00:00     96.0
2017-06-30 21:00:00     90.0
2017-06-30 22:00:00     84.0
2017-06-30 23:00:00     78.0
Freq: h, Name: value, Length: 14592, dtype: float64

In [63]:
len(proc_data.dataset)

14592

In [27]:
train_x, test_x, train_y, test_y = train_test_split(
            proc_data.dataset.index.values, proc_data.dataset['value'], test_size=0.2, shuffle=False)

In [28]:
train_x

array(['2015-11-01T00:00:00.000000000', '2015-11-01T01:00:00.000000000',
       '2015-11-01T02:00:00.000000000', ...,
       '2017-03-01T06:00:00.000000000', '2017-03-01T07:00:00.000000000',
       '2017-03-01T08:00:00.000000000'], dtype='datetime64[ns]')

In [32]:
train_y

date_time
2015-11-01 00:00:00    15.0
2015-11-01 01:00:00    13.0
2015-11-01 02:00:00    10.0
2015-11-01 03:00:00     7.0
2015-11-01 04:00:00     9.0
                       ... 
2017-03-01 04:00:00    42.0
2017-03-01 05:00:00    39.0
2017-03-01 06:00:00    43.0
2017-03-01 07:00:00    43.0
2017-03-01 08:00:00    50.0
Freq: h, Name: value, Length: 11673, dtype: float64

In [29]:
xgb_model = XGBRegressor(eval_metric='mae')

In [33]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder

# Convert datetime to numeric features
def prepare_time_features(df):
    """
    Convert datetime index into useful numeric features for XGBoost
    """
    # Convert to pandas datetime if not already
    df['date_time'] = pd.to_datetime(df['date_time'])
    
    # Extract useful time features
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['year'] = df['date_time'].dt.year
    df['day_of_week'] = df['date_time'].dt.dayofweek
    
    # Drop original datetime column
    return df.drop('date_time', axis=1)

# Prepare your data
train_data = pd.DataFrame({
    'date_time': train_x,
    'value': train_y
})

test_data = pd.DataFrame({
    'date_time': test_x,
    'value': test_y
})

# Create features
train_features = prepare_time_features(train_data)
test_features = prepare_time_features(test_data)

# Separate features and target
X_train = train_features.drop('value', axis=1)
y_train = train_features['value']
X_test = test_features.drop('value', axis=1)
y_test = test_features['value']

# Train model
# xgb_model = XGBRegressor(eval_metric='mae')
# xgb_model.fit(
#     X_train,
#     y_train,
#     eval_set=[(X_train, y_train), (X_test, y_test)],
#     verbose=False
# )

In [53]:
xgb_model.fit(
    X_train,
    y_train,
    # eval_set=[(X_train, y_train), (X_test, y_test)],
    verbose=False
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [55]:
y_train

date_time
2015-11-01 00:00:00    15.0
2015-11-01 01:00:00    13.0
2015-11-01 02:00:00    10.0
2015-11-01 03:00:00     7.0
2015-11-01 04:00:00     9.0
                       ... 
2017-03-01 04:00:00    42.0
2017-03-01 05:00:00    39.0
2017-03-01 06:00:00    43.0
2017-03-01 07:00:00    43.0
2017-03-01 08:00:00    50.0
Freq: h, Name: value, Length: 11673, dtype: float64

In [65]:
def prepare_time_features(dates):
    """Convert datetime series into numeric features for XGBoost"""
    df = pd.DataFrame({'date_time': pd.to_datetime(dates)})
    
    # Extract useful time features
    df['hour'] = df['date_time'].dt.hour
    df['day'] = df['date_time'].dt.day
    df['month'] = df['date_time'].dt.month
    df['year'] = df['date_time'].dt.year
    df['day_of_week'] = df['date_time'].dt.dayofweek
    
    # Return features without datetime column
    return df.drop('date_time', axis=1)

In [66]:
X_full = prepare_time_features(proc_data.dataset.index.values)
y_full = proc_data.dataset['value']

In [67]:
xgb_model.fit(
    X_full,
    y_full,
    verbose=True
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='mae', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [68]:
def predict_future(model, last_date, periods=168):  # default to 1 week (168 hours)
    """
    Generate predictions for future periods
    
    Args:
        model: Trained XGBoost model
        last_date: Last date in training data
        periods: Number of hours to predict into future
    """
    # Generate future dates
    future_dates = pd.date_range(
        start=pd.Timestamp(last_date) + pd.Timedelta(hours=1),
        periods=periods,
        freq='H'
    )
    
    # Prepare features for future dates
    future_features = prepare_time_features(future_dates)
    
    # Make predictions
    predictions = model.predict(future_features)
    
    # Create results dataframe
    results = pd.DataFrame({
        'date_time': future_dates,
        'predicted_value': predictions
    })
    
    return results

# Get last date from your data
last_date = pd.Timestamp(proc_data.dataset.index[-1])

# Make future predictions (e.g., next week)
future_predictions = predict_future(xgb_model, last_date)

C:\Users\krizh\AppData\Local\Temp\ipykernel_19804\3199307643.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_dates = pd.date_range(


In [69]:
future_predictions

,date_time,predicted_value
0,2017-07-01 00:00:00,67.385147
1,2017-07-01 01:00:00,59.374039
2,2017-07-01 02:00:00,48.790020
3,2017-07-01 03:00:00,41.097279
4,2017-07-01 04:00:00,34.455666
...,...,...
163,2017-07-07 19:00:00,89.165237
164,2017-07-07 20:00:00,86.735573
165,2017-07-07 21:00:00,76.780396
166,2017-07-07 22:00:00,70.376259


In [61]:
len(predictions)

2919

In [62]:
predictions

array([58.357044, 74.65956 , 80.03948 , ..., 84.72669 , 79.21824 ,
       70.54479 ], dtype=float32)